## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## Please run all cells until the cell called "Processing project video"
unfortunately the current pipeline does not work neither with the challenge nor the harder challenge videos

In [1]:
#importing libraries and pickle file
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import collections
from IPython.display import HTML
import scipy.spatial
%matplotlib qt




In [2]:
def undistort(img, mtx, dist):
    '''
    method: undistort input image
    input: 
        image to undistort, 
        camera matrix
        distortion coeffs
    output: undistorted image 
    '''
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist  

def binary_sobelx(color_img,kernel_size=3,sx_thresh=(20, 100), gauss_blur=False, gauss_k_size = 3):
    '''
    method: helper method that applies the Sobel operator on the x direction 
    input: 
        original image (in colored format), 
        sobel kernel size, 
        thresholds for the sobel operator (range 0-255)
        flag for applying the gaussian blur
        gaussian blur kernel size (used only if flag is true)
    output:
        binary thresholded copy of the input image with applied sobel operator on the x axis
    '''
    # 1) Convert to grayscale
    gray = cv2.cvtColor(np.copy(color_img), cv2.COLOR_RGB2GRAY)  
    
    # 2) Take both Sobel x and apply gaussian blur
    if gauss_blur:
        smoothed = gaussian_blur(gray,gauss_k_size)
        sobelx = cv2.Sobel(smoothed, cv2.CV_64F, 1, 0,ksize=kernel_size) # Take the derivative in x
    else:
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0,ksize=kernel_size) # Take the derivative in x

    # 3)Absolute x derivative to accentuate lines away from horizontal
    abs_sobelx = np.absolute(sobelx) 
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # 4)Threshold x gradient, apply mask
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    return sxbinary

def sobel_mag_thresh(color_img, kernel_size=3, mag_thresh=(30, 100)):
    '''
    method: helper method that calculates the Sobel gradient 
    magnitue (on both x and y axes), applies to the input 
    image and returns a binary image 
    input: 
        original image (in colored format), 
        sobel kernel size, 
        thresholds for the gradient magniture (range 0-255)
    output:
        binary thresholded copy of the input image with applied sobel operator on both x and y axes
    ''' 
    # 1) Convert to grayscale
    gray = cv2.cvtColor(np.copy(color_img), cv2.COLOR_RGB2GRAY)
    
    # 2) Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=kernel_size)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=kernel_size)
    
    # 3) Calculate the magnitude
    mag = np.sqrt(sobelx**2 + sobely**2)
    
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scale_factor = np.max(mag)/255 
    norm_mag = (mag/scale_factor).astype(np.uint8)     
    
    # 5) Create a binary mask where mag thresholds are met
    binary_mag = np.zeros_like(norm_mag)
    binary_mag[(norm_mag >= mag_thresh[0]) & (norm_mag <=mag_thresh[1])] = 1
    
    return binary_mag

def binary_s_channel(color_img,s_thresh=(150, 255)):
    '''
    method: helper method that gets the saturation channel 
    of an image and generates a binary copy of it using 
    some thresholds in the HLS color space
    input: 
        original image (in colored format),  
        thresholds for the saturation channel (range 0-255)
    output:
        binary thresholded copy of the input image with applied a mask on the saturation channel
    '''   
    #1)make a copy of the image and transform it into HLS space 
    hls = cv2.cvtColor(np.copy(color_img), cv2.COLOR_RGB2HLS)
    #2)get the saturation channel
    s_channel = hls[:,:,2]
    #3)apply masks
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1    
    
    return s_binary

def binary_r_channel(color_img,r_thresh=(200,255)):
    '''
    method: helper method that gets the red channel 
    of an image and generates a binary copy of it using 
    some thresholds in the RGB color space
    input: 
        original image (in colored format),  
        thresholds for the red channel (range 0-255)
    output:
        binary thresholded copy of the input image with applied a mask on the red channel
    ''' 
    #1)copy the image
    img = np.copy(color_img)
    #2) get red channel
    r_channel = img[:,:,0]
    #3)apply masks
    r_binary = np.zeros_like(r_channel)
    r_binary[(r_channel >= r_thresh[0]) & (r_channel <= r_thresh[1])] = 1      
    
    return r_binary 

def binary_filters(s_x = True, bin_sx=0,s_mag=True, bin_s_mag=0, s_ch=True,bin_s_ch=0,r_ch=True,bin_r_ch=0):
    '''
    method: helper method that applies logical OR to all input images  
    input: 
        s_x sobel x flag
        bin_sx sobel x binary image
        s_mag gradient magnitude flag
        bin_s_mag gradient masked binary image
        s_ch saturation channel flag
        bin_s_ch binary-masked-saturation image
        r_ch red channel flag
        bin_r_ch binary-masked-red image
    output:
        binary thresholded copy that is the logical OR of the inputs
    '''     
    #init
    if s_x == True:
        binary_mask = np.zeros_like(bin_sx)
    elif s_mag == True:
        binary_mask = np.zeros_like(bin_s_mag)
    elif s_ch == True:
        binary_mask = np.zeros_like(bin_s_ch)
    elif r_ch == True:
        binary_mask = np.zeros_like(bin_r_ch)

    #apply only specific filters
    if (s_x == True) :
        binary_mask[(bin_sx ==1) | (binary_mask ==1)] = 1                
    if (s_mag == True):
        binary_mask[(bin_s_mag ==1) | (binary_mask ==1)] = 1     
    if (s_ch == True):
        binary_mask[(bin_s_ch ==1) | (binary_mask ==1)] = 1
    if (r_ch == True):
        binary_mask[(bin_r_ch ==1) | (binary_mask ==1)] = 1 
 
    return binary_mask

def colors_and_gradients(input_image):
    '''
    method: apply color filters and gradient operators to the input image.    
    input: 
        input_image
    output:
        binary_or_img image that is the logical OR of different masks
        bin_sobelx binary image with applied sobel operator in the x direction
        bin_s_channel binary image with applied a threshold on the saturation channel
        bin_r_channel binary image with applied a threshold on the red channel
        bin_sobel_mag binary image with applied sobel gradient magnitude thresholds
    '''      
    #apply all filters to the input image
    #input values are the best values found by observing different combinations
    bin_sobelx = binary_sobelx(input_image,kernel_size=5,sx_thresh=(20, 100),gauss_blur=True)
    bin_s_channel = binary_s_channel(input_image,s_thresh=(150, 255))
    bin_r_channel =  binary_r_channel(input_image,r_thresh=(200,255))
    bin_sobel_mag = sobel_mag_thresh(input_image, kernel_size=5, mag_thresh=(30, 100)) 

    #check how many active pixels there are in the red channel and apply a filter.
    #the red channel is very sensitive at times, therefore I filter it out 
    #in some cases
    count_light_in_red_ch =  np.sum(bin_r_channel[:,:])
    
    #the threshold is the result of some visual observations
    #if there are too many activatd pixels in the red channel I exlude it
    if count_light_in_red_ch > 150000:
        binary_or_img = binary_filters(s_x = True, bin_sx = bin_sobelx,
                                   s_mag=True, bin_s_mag = bin_sobel_mag, 
                                   s_ch=True,bin_s_ch=bin_s_channel,
                                   r_ch=False,bin_r_ch=bin_r_channel)
    else:
        binary_or_img = binary_filters(s_x = True, bin_sx = bin_sobelx,
                                   s_mag=True, bin_s_mag = bin_sobel_mag, 
                                   s_ch=True,bin_s_ch=bin_s_channel,
                                   r_ch=True,bin_r_ch=bin_r_channel)
    
    return binary_or_img,bin_sobelx,bin_s_channel,bin_r_channel,bin_sobel_mag

def gaussian_blur(img, kernel_size):
    '''
    method: Applies a Gaussian Noise kernel    
    input: 
        input_image
        kernel size of the operator
    output:
        image with gaussian blur applied
    '''  
    blurred = cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    return blurred



def chessboard_warper(img, nx, ny, mtx, dist):
    '''
    method: Applies a perspective transformation to a warped chessboard 
    image. The edge corners of the final image have an offset from the image edges   
    input: 
        img input_image
        nx number of columns of the chessboard  
        ny number of rows of the chessboard
        mtx camera matrix
        dist camera distortion coeffs
    output:
        unwarped chesshboard (if corners are found in the input image), return 0,0 instead
    '''
    #fix offset
    offset = 100
    
    #image size
    img_size = (img.shape[1], img.shape[0])
    
    #if some corners are found, use the edge corners as reference points
    #for the warp perspective
    ret, corners = cv2.findChessboardCorners(img, (nx,ny), None)
    if ret == True: 

        src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])

        dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                          [img_size[0]-offset, img_size[1]-offset], [offset, img_size[1]-offset]])
        
        #warped, M, invM = warper(img, src, dst)
        M = cv2.getPerspectiveTransform(src, dst)
        warped = cv2.warpPerspective(img,M,img_size,flags=cv2.INTER_LINEAR)
        
        return warped
    else:
        return 0,0

def warper(img, src, dst):
    '''
    method: Applies a warp perspective transformation to any input image        
    input: 
        img input image 
        src coordinate of the warp reference points
        dst coordinates of the warp destination points
    output:
        warped image
        M transformation matrix used to warp the image
        invM inverse transformation matrix (to unwarp the image)
    '''
    # Compute and apply perpective transform
    img_size = (img.shape[1], img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    #warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_NEAREST)  # keep same size as input image
    warped = cv2.warpPerspective(img,M,img_size,flags=cv2.INTER_LINEAR)
    invM = cv2.getPerspectiveTransform(dst,src)
    
    return warped, M, invM
      
def getWarpSrcPts(img):
    '''
    method: get the coordinates of the source perspective transform points 
    from the give image. The hardcoded offset points are the result of several manual test.
    I decided to remove at least almost the entire car hood too. Please note that the order of 
    the output array is P3,P4,P1,P2
    input: 
        img input image 
    output:
        coordinates of 4 source points
    '''
    x_offset_up = 550
    x_offset_low = 170
    y_offset_up = 250
    y_car_hood = 30
    
    #lower left corner of Trapezoid
    P1 = (x_offset_low,img.shape[0]-y_car_hood)
    #upper left corner of Trapezoid
    P2 = (x_offset_up, img.shape[0]-y_offset_up) 
    #upper right corner of Trapezoid
    P3 = (int(round(img.shape[1]-x_offset_up)), img.shape[0]-y_offset_up)
    #lower right corner of Trapezoid
    P4 = (img.shape[1]-x_offset_low,img.shape[0]-y_car_hood)
    
    return  np.float32([P3,P4,P1,P2]) 

def getWarpDstPts(img):
    '''
    method: get the coordinates of the destination perspective transform points 
    from the give image. The hardcoded offset points are the result of several 
    manual tests. Please note that the order of the output array is P3,P4,P1,P2
    input: 
        img input image 
    output:
        coordinates of 4 destination points
    '''
    x_offset = 300
    y_offset_up = 50


    #lower left corner of Trapezoid
    P1 = (x_offset,img.shape[0])
    #upper left corner of Trapezoid
    P2 = (x_offset, y_offset_up) 
    #upper right corner of Trapezoid
    P3 = (img.shape[1]-x_offset, y_offset_up)
    #lower right corner of Trapezoid
    P4 = (img.shape[1]-x_offset,img.shape[0])    
    
    
    return  np.float32([P3,P4,P1,P2])

def getWarpSrcPtsChallenge(img):
    '''
    method: get the coordinates of the source perspective transform points 
    from the give image. The hardcoded offset points are the result of several manual test.
    I decided to remove at least almost the entire car hood too. Please note that the order of 
    the output array is P3,P4,P1,P2
    input: 
        img input image 
    output:
        coordinates of 4 source points
    '''
    x_offset_up_left = 550
    x_offset_up_right = 520
    x_offset_low = 170
    y_offset_up = 230
    y_car_hood = 30
    
    #lower left corner of Trapezoid
    P1 = (x_offset_low,img.shape[0]-y_car_hood)
    #upper left corner of Trapezoid
    P2 = (x_offset_up_left, img.shape[0]-y_offset_up) 
    #upper right corner of Trapezoid
    P3 = (img.shape[1]-x_offset_up_right, img.shape[0]-y_offset_up)
    #lower right corner of Trapezoid
    P4 = (img.shape[1]-x_offset_low,img.shape[0]-y_car_hood)
    
    return  np.float32([P3,P4,P1,P2]) 
      
    
    return  np.float32([P3,P4,P1,P2])
def undistort_and_warp(img,challenge=False):
    '''
    method: applies both undistortion and warp to the input image
    input: 
        img input image 
    output:
        warped image
        M transformation matrix used to warp the image
        invM inverse transformation matrix (to unwarp the image)
        src = source points for the perspective transform
        dst = destination points for the perspective transform 
    '''    
    undist_img = undistort(img, mtx, dist)
    
    if (challenge == False):
        src = getWarpSrcPts(undist_img)
    else:
        src = getWarpSrcPtsChallenge(undist_img)
    dst = getWarpDstPts(undist_img)

    warped,M,invM = warper(undist_img, src, dst)    
    

    return warped,M,invM,src,dst

## Run the calibration process
the calibration process produces coefficients used later in the pipeline

In [3]:
def populate_points(images,objpt,nx,ny):
    '''
    method: generate a list of image points and object points for calibration
    input: 
        images = series of chessboard images
        objpt = coordinates to generate the objpoints
        nx = number of columns in the chessboard
        ny = number of rows in the chessboard
    output:
        objpoints = 3d points in real world space
        imgpoints = 2d points in image plane
    '''     
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane. 
    
    for im in images:
        img = cv2.imread(im)
        imsize = img.shape[1::-1]
        #convert to gray
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        ret, corners = cv2.findChessboardCorners(gray,(nx,ny),None)

        if ret == True:
            objpoints.append(objpt) #always append the same points
            imgpoints.append(corners) #append new corners

    return objpoints,imgpoints

def calibrate(imsize, objpoints, imgpoints):
    '''
    method: calibrate the camera
    input: 
        imsize = size of the image
        objpoints = 3d points in real world space
        imgpoints = 2d points in image plane
    output:
        ret = boolean flag, a value was returned
        mtx = camera matrix
        dist = distortion coefficients
        rvecs = rotation vectors
        tvecs = translation vectors
    '''  
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints,imgpoints,imsize,None,None)
    
    return ret, mtx, dist, rvecs, tvecs


def run_calibration_process():
    '''
    method: run the camera calibration process using a set of chessboard images
    input: 
        none
    output:
        mtx = camera matrix
        dist = distortion coefficients
        rvecs = rotation vectors
        tvecs = translation vectors
    '''      
    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')    

    #assuming all images have the same size
    img = cv2.imread(images[0])
    imsize = img.shape[1::-1]

    nx = 9
    ny = 6

    #build a grid of 3D coordinates
    #x,y coordinates, z remains zero
    objpt = np.zeros((nx*ny,3),np.float32)
    
    #mgrid returns the coordinates values for a given gridsize
    #then we shape the coordinates back in two colums one for x and one for y
    objpt[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2) 

    #populate obj and img points 
    objpoints,imgpoints = populate_points(images,objpt,nx,ny)
    #calibrate the camera
    ret, mtx, dist, rvecs, tvecs = calibrate(imsize, objpoints, imgpoints)
    print("camera calibration done")
    return mtx, dist, rvecs, tvecs

mtx, dist, rvecs, tvecs  = run_calibration_process()

camera calibration done


## Testing: undistort and warp chessboard images as test
please uncomment the last line to visualize the image

In [4]:
def writeup_plot_undistort_warp_chessboard():
    '''
    method: generates one image for the project writeup showing the unwarp process of two 
    chessboard images from the set of the calibration images        
    '''  
    
    #open the first image
    filename = 'camera_cal/calibration3.jpg'
    img1 = mpimg.imread(filename)

    imNum1 = filename.split('camera_cal/calibration')[1].split('.jpg')[0]

    nx = 9
    ny = 6

    undistorted1= undistort(img1, mtx, dist)
    gray1 = cv2.cvtColor(undistorted1, cv2.COLOR_RGB2GRAY)
    top_down1 = chessboard_warper(gray1, nx, ny, mtx, dist)

    #open a second image with different nx and ny
    filename = 'camera_cal/calibration1.jpg'
    img2 = mpimg.imread(filename)

    imNum2 = filename.split('camera_cal/calibration')[1].split('.jpg')[0]

    nx = 9
    ny = 5

    undistorted2 = undistort(img2, mtx, dist)
    gray2 = cv2.cvtColor(undistorted2, cv2.COLOR_RGB2GRAY)
    top_down2 = chessboard_warper(gray2, nx, ny, mtx, dist)


    f, ((ax1, ax2), (ax3,ax4),(ax5,ax6)) = plt.subplots(3,2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img1)
    ax1.set_title('Original calibration image {}'.format(imNum1), fontsize=10)
    ax3.imshow(undistorted1)
    ax3.set_title('Undistorted image {}'.format(imNum1), fontsize=10)
    ax5.imshow(top_down1, cmap='gray')
    ax5.set_title('Undistorted and Warped  image {}'.format(imNum1), fontsize=10)
    ax2.imshow(img2)
    ax2.set_title('Original calibration image {}'.format(imNum2), fontsize=10)
    ax4.imshow(undistorted2)
    ax4.set_title('Undistorted image {}'.format(imNum2), fontsize=10)
    ax6.imshow(top_down2,cmap ='gray')
    ax6.set_title('Undistorted and Warped image {}'.format(imNum2), fontsize=10)


    f.savefig('output_images/undistort_warp_chessboard_examples.jpg')    
#writeup_plot_undistort_warp_chessboard()

## Testing: undistort road images with straight lines for validation
please uncomment the last line to visualize the image


In [21]:
def writeup_plot_undistort_straight_lines():
    '''
    method: generates one image for the project writeup showing the 
    undistort result applied to the two test images with straight lines         
    '''      
    #undistorting straight line images
    test_img_1 = 'test_images/straight_lines1.jpg'
    #test_img_2 = 'test_images/straight_lines2.jpg'
    test_img_2 = 'test_images/test1.jpg'
    img1 = plt.imread(test_img_1)
    img2 = plt.imread(test_img_2)
    test_img_1_undistorted = undistort(img1, mtx, dist)
    test_img_2_undistorted = undistort(img2, mtx, dist)


    f, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize=(18, 7))
    f.tight_layout()
    ax1.imshow(img1)
    ax1.set_title('Original Image 1', fontsize=10)
    ax2.imshow(img2)
    ax2.set_title('Original Image 2', fontsize=10)
    ax3.imshow(test_img_1_undistorted)
    ax3.set_title('Undistorted Image 1', fontsize=10)
    ax4.imshow(test_img_2_undistorted)
    ax4.set_title('Undistorted Image 2', fontsize=10)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.05)

    plt.imsave('output_images/straight_lines1_undist.jpg',test_img_1_undistorted )
    plt.imsave('output_images/straight_lines2_undist.jpg',test_img_2_undistorted )
    
    # Save the full figure...
    f.savefig('output_images/straight_lines_undist.jpg')

#writeup_plot_undistort_straight_lines()

## Testing perspective transform with color images of roads
please uncomment the last line to visualize the image

In [6]:
def writeup_plot_bird_eye_view_straight_lines():
    '''
    method: generates one image for the project writeup showing the 
    undistort and warp result applied to the two test images with straight lines
    the image show the trapezoid shape I chose as reference.
    '''      
    #open first image
    filename1 = 'output_images/straight_lines1_undist.jpg'
    image1 = plt.imread(filename1)
    src = getWarpSrcPts(image1) 
    dst = getWarpDstPts(image1) 

    warped1,M,invM = warper(image1, src, dst)
    plt.imsave('output_images/straight_lines1_warped.jpg',warped1)
    #left reference line warped image
    cv2.line(warped1, (dst[2][0], dst[2][1]), (dst[3][0], dst[3][1]), [0,255,0], 4)
    cv2.circle(warped1, (dst[2][0], dst[2][1]), 2, [0,255,0], 12)
    cv2.circle(warped1, (dst[3][0], dst[3][1]), 2, [0,255,0], 12)

    #right reference line warped image
    cv2.line(warped1, (dst[0][0], dst[0][1]), (dst[1][0], dst[1][1]), [255,0,0], 4)
    cv2.circle(warped1, (dst[0][0], dst[0][1]), 2, [255,0,0], 12)
    cv2.circle(warped1, (dst[1][0], dst[1][1]), 2, [255,0,0], 12)

    #left reference line original image
    cv2.line(image1, (src[2][0], src[2][1]), (src[3][0], src[3][1]), [0,255,0], 4) 
    cv2.line(image1, (src[2][0], src[2][1]), (src[1][0], src[1][1]), [0,0,255], 4)
    cv2.line(image1, (src[3][0], src[3][1]), (src[0][0], src[0][1]), [0,0,255], 4) 
    cv2.circle(image1, (src[2][0], src[2][1]), 2, [0,255,0], 12)
    cv2.circle(image1, (src[3][0], src[3][1]), 2, [0,255,0], 12)

    #right reference line original image
    cv2.line(image1, (src[0][0], src[0][1]), (src[1][0], src[1][1]), [255,0,0], 4)
    cv2.circle(image1, (src[0][0], src[0][1]), 2, [255,0,0], 12)
    cv2.circle(image1, (src[1][0], src[1][1]), 2, [255,0,0], 12)

    filename2 = 'output_images/straight_lines2_undist.jpg'
    image2 = plt.imread(filename2)
    src = getWarpSrcPts(image2) 
    dst = getWarpDstPts(image2) 

    warped2,M,invM = warper(image2, src, dst)

    plt.imsave('output_images/straight_lines2_warped.jpg',warped2)
    #left reference line warped image
    cv2.line(warped2, (dst[2][0], dst[2][1]), (dst[3][0], dst[3][1]), [0,255,0], 4)
    cv2.circle(warped2, (dst[2][0], dst[2][1]), 2, [0,255,0], 12)
    cv2.circle(warped2, (dst[3][0], dst[3][1]), 2, [0,255,0], 12)

    #right reference line warped image
    cv2.line(warped2, (dst[0][0], dst[0][1]), (dst[1][0], dst[1][1]), [255,0,0], 4)
    cv2.circle(warped2, (dst[0][0], dst[0][1]), 2, [255,0,0], 12)
    cv2.circle(warped2, (dst[1][0], dst[1][1]), 2, [255,0,0], 12)

    #left reference line original image
    cv2.line(image2, (src[2][0], src[2][1]), (src[3][0], src[3][1]), [0,255,0], 4) 
    cv2.line(image2, (src[2][0], src[2][1]), (src[1][0], src[1][1]), [0,0,255], 4)
    cv2.line(image2, (src[3][0], src[3][1]), (src[0][0], src[0][1]), [0,0,255], 4) 
    cv2.circle(image2, (src[2][0], src[2][1]), 2, [0,255,0], 12)
    cv2.circle(image2, (src[3][0], src[3][1]), 2, [0,255,0], 12)

    #right reference line original image
    cv2.line(image2, (src[0][0], src[0][1]), (src[1][0], src[1][1]), [255,0,0], 4)
    cv2.circle(image2, (src[0][0], src[0][1]), 2, [255,0,0], 12)
    cv2.circle(image2, (src[1][0], src[1][1]), 2, [255,0,0], 12)


    f, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize=(18, 7))
    f.tight_layout()
    ax1.imshow(image1)
    ax1.set_title('Original image 1 with reference lines and points', fontsize=10)
    ax2.imshow(image2)
    ax2.set_title('Original image 2 with reference lines and points', fontsize=10)
    ax3.imshow(warped1)
    ax3.set_title('Img 1 Undistorted and warped. \nThe lines should be parallel in the warped space.', fontsize=10)

    ax4.imshow(warped2)
    ax4.set_title('Img2 Undistorted and warped. \nThe lines should be parallel in the warped space.', fontsize=10)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.05,wspace = None,hspace = 0.3)


    # Save the full figure...
    f.savefig('output_images/warped_straight_lines.jpg')


#writeup_plot_bird_eye_view_straight_lines()


## Test unwarp paramenters for challenge video

In [7]:
def writeup_plot_bird_eye_view_straight_lines_challenge():
    '''
    method: generates one image for the project writeup showing the 
    undistort and warp result applied to one frame from the challenge video
    in fact it seems that the reference points I chose for the project video
    do not work well for the challenge video
    '''      
    
    filename = 'test_images/challenge_video/test373.jpg'

    img = plt.imread(filename)

    image1 = undistort(img, mtx, dist)

    src = getWarpSrcPtsChallenge(image1)
    #src = getWarpSrcPts(image1) 
    dst = getWarpDstPts(image1) 

    warped1,M,invM = warper(image1, src, dst)
    #plt.imsave('output_images/straight_lines1_warped.jpg',warped1)
    #left reference line warped image
    cv2.line(warped1, (dst[2][0], dst[2][1]), (dst[3][0], dst[3][1]), [0,255,0], 4)
    cv2.circle(warped1, (dst[2][0], dst[2][1]), 2, [0,255,0], 12)
    cv2.circle(warped1, (dst[3][0], dst[3][1]), 2, [0,255,0], 12)

    #right reference line warped image
    cv2.line(warped1, (dst[0][0], dst[0][1]), (dst[1][0], dst[1][1]), [255,0,0], 4)
    cv2.circle(warped1, (dst[0][0], dst[0][1]), 2, [255,0,0], 12)
    cv2.circle(warped1, (dst[1][0], dst[1][1]), 2, [255,0,0], 12)

    #left reference line original image
    cv2.line(image1, (src[2][0], src[2][1]), (src[3][0], src[3][1]), [0,255,0], 4) 
    cv2.line(image1, (src[2][0], src[2][1]), (src[1][0], src[1][1]), [0,0,255], 4)
    cv2.line(image1, (src[3][0], src[3][1]), (src[0][0], src[0][1]), [0,0,255], 4) 
    cv2.circle(image1, (src[2][0], src[2][1]), 2, [0,255,0], 12)
    cv2.circle(image1, (src[3][0], src[3][1]), 2, [0,255,0], 12)

    #right reference line original image
    cv2.line(image1, (src[0][0], src[0][1]), (src[1][0], src[1][1]), [255,0,0], 4)
    cv2.circle(image1, (src[0][0], src[0][1]), 2, [255,0,0], 12)
    cv2.circle(image1, (src[1][0], src[1][1]), 2, [255,0,0], 12)

   
    f, (ax1,ax2) = plt.subplots(1, 2, figsize=(18, 7))
    f.tight_layout()
    ax1.imshow(image1)
    ax1.set_title('Original image 1 with reference lines and points', fontsize=10)

    
    ax2.imshow(warped1)
    ax2.set_title('Undistorted and warped image. \nThe lines should be parallel in the warped space.', fontsize=10)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.05,wspace = None,hspace = 0.3)


    # Save the full figure...
    #f.savefig('output_images/warped_straight_lines_challenge.jpg')


#writeup_plot_bird_eye_view_straight_lines_challenge()


## Testing: apply color, gradient, undistort and warp to other test images (curved lines)
please uncomment the last line to visualize the images (the amount of output images depends on the folder you choose for testing. If no folder is given as input (folder_to_test = '') then the default folder is "test_images" (where I have added some additional images with respect to the original 6). Other possible folders are "project_video","straight_lines" and "challenge_video" 

In [8]:
def writeup_test_colors_and_gradiends(folder=''):
    '''
    method: generates several images. This method is used for testing and for the 
    project writeup showing the output of all single color filters and gradients
    but also the combined figure (logical OR of single channels) that will be used
    later in the final pipeline
    '''    
    
    root_path = 'test_images/'
    path = root_path + folder

    full_path = path + '52.jpg'


    images = glob.glob(full_path)    

    for im in images:
        imNum = im.split(path)[1].split('.jpg')[0]
        image = mpimg.imread(im)

        warped,M,invM,src,dst = undistort_and_warp(image,challenge=True)
        binary_or_img,bin_sobelx,bin_s_channel,bin_r_channel,bin_sobel_mag = colors_and_gradients(warped)


        #convert the image to three channel for vis purposes
        binary_or_img_color = cv2.cvtColor(binary_or_img, cv2.COLOR_GRAY2RGB) *255
        #left reference line warped image
        cv2.line(binary_or_img_color, (dst[2][0], dst[2][1]), (dst[3][0], dst[3][1]), [0,255,0], 4)
        #right reference line warped image
        cv2.line(binary_or_img_color, (dst[0][0], dst[0][1]), (dst[1][0], dst[1][1]), [255,0,0], 4)    

        f, ((ax1,ax2,ax3),(ax4,ax5,ax6)) = plt.subplots(2, 3, figsize=(18, 7))
        f.tight_layout()
        ax1.imshow(warped)
        ax1.set_title('Warped test image {}'.format(imNum), fontsize=10)
        ax2.imshow(cv2.cvtColor(bin_s_channel, cv2.COLOR_GRAY2RGB) *255)
        ax2.set_title('Binary saturation channel', fontsize=10)
        ax3.imshow(cv2.cvtColor(bin_r_channel, cv2.COLOR_GRAY2RGB) *255)
        ax3.set_title('Binary red channel', fontsize=10)
        ax4.imshow(cv2.cvtColor(bin_sobelx, cv2.COLOR_GRAY2RGB) *255)
        ax4.set_title('Binary sobelx channel', fontsize=10)
        ax5.imshow(cv2.cvtColor(bin_sobel_mag, cv2.COLOR_GRAY2RGB) *255)
        ax5.set_title('Binary sobel magnitude (x and y) channel', fontsize=10)
        ax6.imshow(binary_or_img_color)
        ax6.set_title('Binary OR of other channels \n(red channel might be filtered out)', fontsize=10)    
        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.05,wspace = None,hspace = 0.3)
        
        #f.savefig('output_images/colors_gradients_examples{}.jpg'.format(imNum))

folder_to_test = ''
#folder_to_test  = 'project_video/test'
#folder_to_test ='straight_lines/test'
folder_to_test ='challenge_video/test'
#writeup_test_colors_and_gradiends(folder_to_test)

## Finding lane pixels with sliding windows and polynomials

In [18]:
def build_poly_lines(img_shape, left_fit,right_fit):
    '''
    method: build two vertical lines using the the y axis of as input coordinates 
    and the two polynomial coefficients. The coefficients must be three per line
    (the method expects 2 deg polynomial coeffs). An exception is catched if the 
    polynomial coeffs are none or incorrect
    input: 
        img input image 
        left_fit = polynomial coefficients for the left line
        right_fit = polynomial coefficients for the right line
    output:
        left_fitx = x coordinates of the left line
        right_fitx = x coordinates of the right line
        ploty y coordinates (the same for both lines)
    '''   
    
    # Generate y values for line evaluation
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    
    #evaluate the lines using the coeffs
    try:           
        right_fitx = ploty**2*right_fit[0] + ploty*right_fit[1] + right_fit[2]
        left_fitx = ploty**2*left_fit[0] + ploty*left_fit[1] + left_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    return left_fitx, right_fitx, ploty

def fit_poly(leftx, lefty, rightx, righty,xm_per_pix,ym_per_pix):
    '''
    method: generates the second degree polynomial 
    coefficients for two set of points
    input: 
        leftx = x coord of the first line
        lefty = y coord of the first line
        rightx = x coord of the second line
        righty = y coord of the second line
        xm_per_pix = pix to meters conversion  
        ym_per_pix = pix to meters conversion 
    output:
        left_fit = 2 degree poly coefficients for first line
        right_fit = 2 degree poly coefficients for second line 
        left_fit_meters = 2 degree poly coefficients for first line converted
        right_fit_meters = = 2 degree poly coefficients for second line converted
        return -1,-1,-1,-1 if any of the input vectors is empty 
    ''' 
    #build result dictionary
    result = {'left_fit':-1,'right_fit':-1,'left_fit_meters':-1,'right_fit_meters':-1}

    if (lefty.size>0 and leftx.size>0 and righty.size>0 and rightx.size>0):
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

        left_fit_meters = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
        right_fit_meters = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

        result['left_fit'] = left_fit
        result['right_fit'] = right_fit
        result['left_fit_meters'] = left_fit_meters
        result['right_fit_meters'] = right_fit_meters
         
    
    return result


def lane_search_with_windows(binary_image, xm_per_pix, ym_per_pix, bottom_half = False):
    '''
    method: takes an imput image, searches for road lanes (left and righ lane)
    using a computational intensive sliding window method. 
    If the flag bottom_half is true, the image searches for lines in the bottom
    half of the image only
    input: 
        binary_image input image should be a binary image
        xm_per_pix = pix to meters conversion  
        ym_per_pix = pix to meters conversion        
        bottom_half flag, Flase by default
    output:
        out_img input image with marked lines and windows
        left_fitx x coordinats of the left line
        right_fitx x coordinats of the right line
        ploty y coordinats of the both lines line
        left_fit 2deg polynomial coeffs of the left line
        right_fit 2deg polynomial coeffs of the right line
        left_fit_meters = poly coeffs converted in meters for curvature measure
        right_fit_meters = poly coeffs converted in meters for curvature measure
        return -1,-1,-1,-1,-1,-1,-1,-1 in case of invalid results
    '''  
    result = {'out_img':-1,
             'left_fitx':-1,
             'right_fitx':-1,
             'ploty':-1,
             'left_fit':-1,
             'right_fit':-1,
             'left_fit_meters':-1,
             'right_fit_meters':-1,}
    
    # Find our lane pixels first (find both lanes using the sliding windows method)
    # the result is the coordinates of both lanes
    # the method returns also the output image (a binary image) that 
    # shows  the two lanes and the windows
    leftx, lefty, rightx, righty, out_img = lane_search_vertical_windows(binary_image,bottom_half)

    #Fit a second order polynomial to each using `np.polyfit` 
    fitted_poly = fit_poly(leftx, lefty, rightx, righty,xm_per_pix,ym_per_pix)
    
    left_fit = fitted_poly['left_fit']
    right_fit = fitted_poly['right_fit']
    left_fit_meters = fitted_poly['left_fit_meters']
    right_fit_meters = fitted_poly['right_fit_meters']

    #check if the poly coeffs are valid 
    if (type(left_fit) != int and left_fit.all() != -1):
        #consider full image or only bottom half of it
        if (bottom_half==False):
            imsize = (binary_image.shape[0]-1, binary_image.shape[0])
        else:
            imsize = (binary_image.shape[0]-1, binary_image.shape[0]//2)

        # Generate x and y values for plotting
        left_fitx, right_fitx, ploty =  build_poly_lines(binary_image.shape, left_fit,right_fit)

        ## Visualization ##
        # Colors in the left and right lane regions
        out_img[lefty, leftx] = [255, 0, 0]
        out_img[righty, rightx] = [0, 0, 255]

        result['out_img'] = out_img
        result['left_fitx'] = left_fitx
        result['right_fitx'] = right_fitx
        result['ploty'] = ploty
        result['left_fit'] = left_fit
        result['right_fit'] = right_fit
        result['left_fit_meters'] = left_fit_meters
        result['right_fit_meters'] =  right_fit_meters     
        
    return result
        
    
def lane_search_around_poly(binary_image,left_poly_fit,right_poly_fit,xm_per_pix, ym_per_pix):
    '''
    method: takes an imput image, searches for road lanes (left and righ lane)
    around two given polynomials. The polynomials are expressed as poly coefficients
    input: 
        binary_image = input image in binary format
        left_poly_fit = coeffs of the "left" polynomial
        right_poly_fit = coeffs of the "right" polynomial
    output:
        image_with_lanes = output image
        left_fitx = x coords of the left lane line
        right_fitx = x coords of the right lane line
        ploty = y coords of both lane lines
        left_fit = poly coeffs of the detected left lane line
        right_fit = poly coeffs of the detected right lane line
        left_fit_meters = poly coeffs converted in meters for curvature measure
        right_fit_meters = poly coeffs converted in meters for curvature measure
        return -1,-1,-1,-1,-1,-1,-1,-1 in case of invalid results
    '''
    result = {'image_with_lanes':-1,
             'left_fitx':-1,
             'right_fitx':-1,
             'ploty':-1,
             'left_fit':-1,
             'right_fit':-1,
             'left_fit_meters':-1,
             'right_fit_meters':-1,}   
    
    # HYPERPARAMETERS
    margin = 30

    # Grab activated pixels from the binary image given as input
    nonzero = binary_image.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    #search for the indices of activated pixels around the two polys
    left_lane_inds = ((nonzerox <= ((nonzeroy**2)*left_poly_fit[0]+nonzeroy*left_poly_fit[1]+left_poly_fit[2])+margin)&
                      (nonzerox > ((nonzeroy**2)*left_poly_fit[0]+nonzeroy*left_poly_fit[1]+left_poly_fit[2])-margin))
    
    right_lane_inds = ((nonzerox <= ((nonzeroy**2)*right_poly_fit[0]+nonzeroy*right_poly_fit[1]+right_poly_fit[2])+margin)&
                      (nonzerox > ((nonzeroy**2)*right_poly_fit[0]+nonzeroy*right_poly_fit[1]+right_poly_fit[2])-margin))
    
    # extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]


    # Fit new polynomials
    fitted_poly = fit_poly(leftx, lefty, rightx, righty,xm_per_pix,ym_per_pix)
    
    left_fit = fitted_poly['left_fit']
    right_fit = fitted_poly['right_fit']
    left_fit_meters = fitted_poly['left_fit_meters']
    right_fit_meters = fitted_poly['right_fit_meters']
    
    #check if the poly coeffs are valid 
    if (type(left_fit) != int and left_fit.all() != -1):
        #build new lane lines
        left_fitx, right_fitx, ploty = build_poly_lines(binary_image.shape, left_fit,right_fit)

        ## Visualization ##
        # Create an image to draw on and an image to show the selection window
        out_img = np.dstack((binary_image, binary_image, binary_image))*255
        window_img = np.zeros_like(out_img)

        # Color in left and right line pixels
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                  ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))

        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        image_with_lanes = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
        
        result['image_with_lanes'] = out_img
        result['left_fitx'] = left_fitx
        result['right_fitx'] = right_fitx
        result['ploty'] = ploty
        result['left_fit'] = left_fit
        result['right_fit'] = right_fit
        result['left_fit_meters'] = left_fit_meters
        result['right_fit_meters'] =  right_fit_meters 
        
    return result


def measure_curvature_in_meters(ym_per_pix,xm_per_pix,ploty,left_poly_fit,right_poly_fit):
    '''
    method: Calculates the curvature of polynomial functions in meters.
    input: 
        ym_per_pix = conversion rate for y axis
        xm_per_pix = conversion rate for x axis
        ploty = y coords of the lane lines
        left_poly_fit = poly coeffs left lane line
        right_poly_fit = poly coeffs right lane line
    output:
        left_curverad = radius of left curve in meters
        right_curverad = radius of right curve in meters
    '''   
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    #calculation of R_curve (radius of curvature) for each line
    left_curverad = ((1 + (2*left_poly_fit[0]*y_eval*ym_per_pix + left_poly_fit[1])**2)**1.5) / np.absolute(2*left_poly_fit[0])
    right_curverad = ((1 + (2*right_poly_fit[0]*y_eval*ym_per_pix + right_poly_fit[1])**2)**1.5) / np.absolute(2*right_poly_fit[0])
    
    return left_curverad, right_curverad

def lane_search_vertical_windows(binary_image,bottom_half=False):
    '''
    method: apply the vertical sliding windows process to find 
    lane lines within the given input image (that should be a binary 
    image). If the flag is true, the vertical windows stop at half of 
    the image.
    input: 
        binary_image = input image
        bottom_half = flag, if true the algo search only within the bottom
                    half of the image (which is the part of the image closest 
                    to the camera)
    output:
        leftx = x coords of the left lane line
        lefty = y coords of the left lane line
        rightx = x coords of the right lane line
        righty = y coords of the right lane line
        out_img = output image with lanes marked
    '''
    
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_image[binary_image.shape[0]//2:,:], axis=0)
    
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_image, binary_image, binary_image))
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 20
    # Set the width of the windows +/- margin
    margin = 30
    # Set minimum number of pixels found to recenter window
    minpix = 30

    # Set height of windows - based on nwindows above and image shape
    # if the flag is true, the process should stop at half of the image
    # (with respect to the y axis)
    if (bottom_half==False):
        window_height = np.int(binary_image.shape[0]//nwindows)
    else:
        window_height = np.int((binary_image.shape[0]//2)//nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_image.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_image.shape[0] - (window+1)*window_height
        win_y_high = binary_image.shape[0] - window*window_height
        
        #Find the four below boundaries of the window
        win_xleft_low = leftx_current - margin  
        win_xleft_high = leftx_current + margin  
        
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzerox >= win_xleft_low) & (nonzerox <win_xleft_high) &
        (nonzeroy>=win_y_low) & (nonzeroy <win_y_high)).nonzero()[0]
        good_right_inds = ((nonzerox >=win_xright_low)&(nonzerox <win_xright_high)&
                           (nonzeroy>=win_y_low) & (nonzeroy <win_y_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        #If you found > minpix pixels, recenter next window
        #(`right` or `leftx_current`) on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

def car_lanes_offset(left,right,imwidth):
    '''
    method: calculate the car offset with respect to the lanes.
    Positive offsets means that the car is on the left of the 
    middle lane
    input: 
        left = x reference coordinate from the left lane
        right = x reference coordinate from the right lane
        imwidth = width of the image
    output:
        offset = car offset
        lane_width = lane width
    '''
    
    #the middle point of the image is half of the image width
    imMid = imwidth//2
    #calculate the middle point between lanes
    laneMid = left+(right-left)//2
    #width of the lane
    lane_width = right-left

    #calculate offset with respect to the middle of the image (which
    # corresponds ot the position of the camera)
    offset = laneMid-imMid
    return offset,lane_width

def color_driving_space(grayImg,left_fitx, right_fitx, ploty):
    '''
    method: use cv2.fillpoly to color the driving space between two lines.
    The driving space is the area in front of the car between the detected lines.
    input: 
        grayImg = input image
        left_fitx = x coords of the left lane line
        right_fitx = x coords of the right lane line
        ploty = y coords of both lane lines
    output:
        driving_space = image with colored driving space inlcuding the two lines
    '''
    #get height and width of the image
    h, w = grayImg.shape[:2]

    #get the "driving space" between the lines and the points of the lines themselves
    pointsL = np.array([None])
    pointsR = np.array([None])
    pointsL = np.array([[[xi, yi]] for xi, yi in zip(left_fitx, ploty) if (0<=xi<w and 0<=yi<h)]).astype(np.int32)
    pointsR = np.array([[[xi, yi]] for xi, yi in zip(right_fitx, ploty) if (0<=xi<w and 0<=yi<h)]).astype(np.int32)
    pointsR = np.flipud(pointsR)
    #driving space points
    points = np.concatenate((pointsL, pointsR))

    driving_space = grayImg.copy()
    driving_space = cv2.cvtColor(driving_space,cv2.COLOR_GRAY2RGB)
    #color driving space
    cv2.fillPoly(driving_space, [points], color=[0,255,0])
    #add left line overlay
    cv2.polylines(driving_space, [pointsL], color=[255,0,0], isClosed = False,thickness = 20)
    #add right line overlay
    cv2.polylines(driving_space, [pointsR], color=[255,0,0], isClosed = False,thickness = 20)   
    
    return driving_space

def warp_binary_img(image,challenge=False):
    '''
    method: apply undistort, perspective transform, 
    color filtering and gradients to the input image
    input: 
        image = input image to process
    output:
        warped = undistorted and warped image
        M = perspective transform matrix
        invM = inverse perspective transform matrix
        binary_or_img = binary image
    '''    
    #apply undistort and perspective transform
    warped,M,invM,src,dst = undistort_and_warp(image,challenge)
    #apply color filters and gradients
    binary_or_img,bin_sobelx,bin_s_channel,bin_r_channel,bin_sobel_mag = colors_and_gradients(warped)
    
    return warped,M,invM,binary_or_img


def add_tags(image,leftcurv,rightcurv,left,right,xm_per_pix,testing=False):
    '''
    method: helper method to add text to the input image.     
    input: 
        image = input image to process
        testing = if true, show more data than needed for final project
        leftcurv = curvature radius of left line
        rightcurv = curvature radius of right line
        left = x coord of one point on the left lane line
        right = x coord of one point on the right lane line
    output:
        none
    '''    
    car_offset,laneWidth = car_lanes_offset(left,right,image.shape[1])
    leftTxt = "Left curvr:{:.2f}[m]".format(leftcurv)    
    rigtTxt = "Right curvr:{:.2f}[m]".format(rightcurv)
    avgTxt = "Average curvr:{:.2f}[m]".format((leftcurv+rightcurv)/2)
    if car_offset>0:
        offsetTxt = "Car offset:{:.2f}[m] to the left".format(car_offset*xm_per_pix)    
    elif car_offset<0:
        offsetTxt = "Car offset:{:.2f}[m] to the right".format(abs(car_offset*xm_per_pix))    
    else:
        offsetTxt = "Car offset:{:.2f}[m]".format(car_offset*xm_per_pix)    
    
    widthtTxt ="Lane Width:{:.2f}[m]".format(laneWidth*xm_per_pix)
    
    position = (400,100)
    cv2.putText(
         image, #numpy array on which text is written
         avgTxt, #text
         position, #position at which writing has to start
         cv2.FONT_HERSHEY_SIMPLEX, #font family
         1, #font size
         (255, 255, 255, 255), #font color
         3) #font stroke 
    position = (400,140)
    cv2.putText(
         image, #numpy array on which text is written
         offsetTxt, #text
         position, #position at which writing has to start
         cv2.FONT_HERSHEY_SIMPLEX, #font family
         1, #font size
         (255, 255, 255, 255), #font color
         3) #font stroke 
    if (testing):
        position = (400,180)    
        cv2.putText(
             image, #numpy array on which text is written
             leftTxt, #text
             position, #position at which writing has to start
             cv2.FONT_HERSHEY_SIMPLEX, #font family
             1, #font size
             (255, 255, 255, 255), #font color
             3) #font stroke 
        position = (400,220)  
        cv2.putText(
             image, #numpy array on which text is written
             rigtTxt, #text
             position, #position at which writing has to start
             cv2.FONT_HERSHEY_SIMPLEX, #font family
             1, #font size
             (255, 255, 255, 255), #font color
             3) #font stroke 
        position = (400,260)
        cv2.putText(
             image, #numpy array on which text is written
             widthtTxt, #text
             position, #position at which writing has to start
             cv2.FONT_HERSHEY_SIMPLEX, #font family
             1, #font size
             (255, 255, 255, 255), #font color
             3) #font stroke 

 
def averaging_poly(lines):
    '''
    method: calculate the average of the poly coeff of the two lane lines
    using the lines in the buffer
    input: 
        lines (tuple of two lines)
    output:
        avg_left = left line as average of the poly coeffs
        avg_right = right line as average of the poly coeffs
    '''     
    left_coeffs = []
    avg_left = []
    right_coeffs = []
    avg_right = []
    
    #unpack lines and extract values
    for l,r in lines:
        left_coeffs.append(l.getLatestFit())
        right_coeffs.append(r.getLatestFit())
        
    #calculate means
    avg_left = np.mean(np.stack(left_coeffs), axis=0)
    avg_right = np.mean(np.stack(right_coeffs), axis=0)

    return avg_left, avg_right

def averaging_curvatures(lines):
    '''
    method: calculate the average curvatures of each line using
    the lines in the buffer    
    input: 
        lines (tuple of two lines)
    output:
        avg_left_cr = average curvature radius of left line
        avg_right_cr = average curvature radius of right line
    '''     
    left_cr = []
    avg_left = []
    right_cr = []
    avg_right = []
    
    #unpack lines and extract values
    for l,r in lines:
        left_cr.append(l.getCurvature())
        right_cr.append(r.getCurvature())
        
    #calculate means
    avg_left = np.mean(np.stack(left_cr), axis=0)
    avg_right = np.mean(np.stack(right_cr), axis=0)

    return avg_left, avg_right

class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 

        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
    
    def setDetectedFlag(self, flag):
        self.detected = flag
        
    def getDetectedFlag(self):
        return self.detected
        
    def setLatestFitx(self,fitx):
        self.recent_xfitted = fitx
        
    def getLatestFitx(self):
        return self.recent_xfitted 
    
    def setLatestFit(self,fit_coeffs):
        self.current_fit = fit_coeffs
        
    def getLatestFit(self):
        return self.current_fit     

    def setCurvature(self,curv):
        self.radius_of_curvature = curv
        
    def getCurvature(self):
        return self.radius_of_curvature    

def getDequeData(d):
    return [data for data in d]

def get_last_lines(d):
    return d[-1]

def process_frame(image):
    
    '''
    method: main method to process video frames
    input: 
        image = current frame to process
    output:
        procesed_image 
    '''    
    #use a global variable for a lane buffer
    global LinesDeque

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 15/720 # meters per pixel in y dimension based on visual observation
    xm_per_pix = 3.7/600 # meters per pixel in x dimension based on visual observation
    
    
    #get bird eye view of the frame
    warped,M,invM,binary_warped = warp_binary_img(image)

    #variables
    valid_l_fit_x = None
    valid_l_cr = None
    valid_r_fit_x = None
    valid_r_cr = None

    # if the buffer is full we can start smoothing
    # lines based on the last detected lanes. The buffer is a ring
    # buffer so that older fralines are always replaced by recent ones
    if len(LinesDeque) == max_queue_size:

        # get the last poly values (averaged across 
        # the lines in the buffer)
        lines = getDequeData(LinesDeque)
        left_avg_coeffs,right_avg_coeffs = averaging_poly(lines)
        left_avg_cr,right_avg_cr = averaging_curvatures(lines)
         
        #use the poly coeffs from last line to search new lines
        find_lanes = lane_search_around_poly(binary_warped,left_avg_coeffs,right_avg_coeffs,xm_per_pix, ym_per_pix)

        result = find_lanes['image_with_lanes']
        
        #check if new line exists
        if (type(result) != int and result.all() != -1):
            left_fitx = find_lanes['left_fitx']
            right_fitx = find_lanes['right_fitx']
            ploty = find_lanes['ploty']
            left_fit = find_lanes['left_fit']
            right_fit = find_lanes['right_fit']
            lfit_meters = find_lanes['left_fit_meters']
            rfit_meters = find_lanes['right_fit_meters']            
            
            #decide if the new polynomial is close enough to the previous lines
            coeff_left = [left_avg_coeffs,left_fit]
            coeff_right = [right_avg_coeffs,right_fit]
                       
            coeff_dists_left = scipy.spatial.distance_matrix(coeff_left, coeff_left)
            coeff_dists_right = scipy.spatial.distance_matrix(coeff_right, coeff_right)
            #calculate difference from average lines
            poly_difference = np.mean([coeff_dists_left[0][1],coeff_dists_right[0][1]])

            #calculate offset and lanewidth
            offset,lanewidth = car_lanes_offset(left_fitx[-1],right_fitx[-1],warped.shape[1])
            
        else:
            #if current line does not exist, use last one
            last_left,last_right = get_last_lines(LinesDeque)
                      
            #decide if the new polynomial is close enough to the previous lines
            coeff_left = [left_avg_coeffs,last_left.getLatestFit()]
            coeff_right = [right_avg_coeffs,last_right.getLatestFit()]           
            
            coeff_dists_left = scipy.spatial.distance_matrix(coeff_left, coeff_left)
            coeff_dists_right = scipy.spatial.distance_matrix(coeff_right, coeff_right)
            
            #calculate difference from average lines
            poly_difference = np.mean([coeff_dists_left[0][1],coeff_dists_right[0][1]])
            
            poly_fitx_last_left = last_left.getLatestFitx()
            poly_fitx_last_right = last_right.getLatestFitx()
            
            #calculate offset and lanewidth
            offset,lanewidth = car_lanes_offset(poly_fitx_last_left[-1],poly_fitx_last_right[-1],warped.shape[1])           
            
            #get values form last line
            left_fitx =  poly_fitx_last_left
            right_fitx = poly_fitx_last_right
            left_fit = last_left.getLatestFit()
            right_fit = last_right.getLatestFitx()

        
        
        
        #conversions
        lanewidth = lanewidth*xm_per_pix
        offset = offset*xm_per_pix
        
        #all conditions must be satisfied to accept the line
        lanewidth_condition = lanewidth >= 3.3 and lanewidth < 4.2
        offset_condition = abs(offset) < 0.45
        poly_condition = poly_difference < 30

        if  lanewidth_condition and offset_condition and poly_condition:
            #good enough
            valid_l_fit_x = left_fitx
            valid_r_fit_x = right_fitx
            valid_left_fit = left_fit
            valid_right_fit = right_fit
            left=Line()
            right=Line()
            left.setDetectedFlag(False)
            right.setDetectedFlag(False)
            
        else:
            #else empty the buffer, search line using sliding windows and append it
            LinesDeque.clear()
            #try detecting the lane on the bottom of the image
            find_lanes = lane_search_with_windows(binary_warped,xm_per_pix,ym_per_pix,bottom_half=False)

            result = find_lanes['out_img']
            valid_l_fit_x = find_lanes['left_fitx']
            valid_r_fit_x = find_lanes['right_fitx']
            ploty = find_lanes['ploty']
            valid_left_fit = find_lanes['left_fit']
            valid_right_fit = find_lanes['right_fit']
            lfit_meters = find_lanes['left_fit_meters']
            rfit_meters = find_lanes['right_fit_meters']
            
            left=Line()
            right=Line()
            left.setDetectedFlag(True)
            right.setDetectedFlag(True)
          
    else:
        #fill the buffer with lines detected with the sliding windows method
        find_lanes = lane_search_with_windows(binary_warped,xm_per_pix,ym_per_pix,bottom_half=False)

        result = find_lanes['out_img']
        valid_l_fit_x = find_lanes['left_fitx']
        valid_r_fit_x = find_lanes['right_fitx']
        ploty = find_lanes['ploty']
        valid_left_fit = find_lanes['left_fit']
        valid_right_fit = find_lanes['right_fit']
        lfit_meters = find_lanes['left_fit_meters']
        rfit_meters = find_lanes['right_fit_meters']
        
        left=Line()
        right=Line()
        left.setDetectedFlag(True)
        right.setDetectedFlag(True)
    
    #measure curvature radius
    valid_l_cr, valid_r_cr = measure_curvature_in_meters(ym_per_pix,xm_per_pix,ploty,lfit_meters,rfit_meters)
    
    #update the line buffer with new data                                                                                                                                    
    left.setLatestFitx(valid_l_fit_x)
    left.setLatestFit(valid_left_fit)
    left.setCurvature(valid_l_cr)
    right.setLatestFitx(valid_r_fit_x)
    right.setLatestFit(valid_right_fit)
    right.setCurvature(valid_r_cr)    
    LinesDeque.append((left,right))
                                                                                                                                          
    # prepare visualization
    warpedGray = cv2.cvtColor(warped,cv2.COLOR_RGB2GRAY)
    #use valid lines 
    driving_space = color_driving_space(warpedGray,valid_l_fit_x, valid_r_fit_x, ploty)
    img_size = (driving_space.shape[1], driving_space.shape[0])
                                                                                                                                          
    #warp the image back into the original image space           
    unwarped = cv2.warpPerspective(driving_space,invM,img_size,flags=cv2.INTER_LINEAR)    
    #merge driving space into original image
    procesed_image = cv2.addWeighted(image, 1, unwarped, 0.3, 0)
    #add tags to the image 
    add_tags(procesed_image,valid_l_cr,valid_r_cr,valid_l_fit_x[-1], valid_r_fit_x[-1],xm_per_pix,testing=False)
    
    return procesed_image


  



## Testing:  pipeline on warped test images

please uncomment the last line to visualize the images (the amount of output images depends on the folder you choose for testing. If no folder is given as input (folder_to_test = '') then the default folder is "test_images" (where I have added some additional images with respect to the original 6). Other possible folders are "project_video","straight_lines" and "challenge_video" 

In [10]:
def writeup_test_pipeline_with_images(folder=''):
    '''
    method: test  several images. This method is used for testing 
    and for the project writeup showing the output of the lane 
    finding approaches on some key images for this project    
    '''    
    
    root_path = 'test_images/'
    path = root_path + folder

    full_path = path + '*.jpg'
    
    images = glob.glob(full_path)    

    for im in images:

        imNum = im.split(path)[1].split('.jpg')[0]

        image = plt.imread(im)
        # Define conversions in x and y from pixels space to meters
        ym_per_pix = 15/720 # meters per pixel in y dimension based on visual observation
        xm_per_pix = 3.7/600 # meters per pixel in x dimension based on visual observation
        
        warped,M,invM,src,dst = undistort_and_warp(image,challenge=True)
        binary_or_img,bin_sobelx,bin_s_channel,bin_r_channel,bin_sobel_mag = colors_and_gradients(warped)

        #lineImg1,left_fitx, right_fitx, ploty,left_fit, right_fit = lane_search_around_poly(binary_or_img,left_fit_prev,right_fit_prev)
        find_lanes = lane_search_with_windows(binary_warped,xm_per_pix,ym_per_pix,bottom_half=False)

        lineImg1 = find_lanes['out_img']
        valid_l_fit_x = find_lanes['left_fitx']
        valid_r_fit_x = find_lanes['right_fitx']
        ploty = find_lanes['ploty']
        valid_left_fit = find_lanes['left_fit']
        valid_right_fit = find_lanes['right_fit']
        lfit_meters = find_lanes['left_fit_meters']
        rfit_meters = find_lanes['right_fit_meters']
        

        # Calculate the radius of curvature in meters for both lane lines
        valid_l_cr, valid_r_cr = measure_curvature_in_meters(ym_per_pix,xm_per_pix,ploty,lfit_meters,rfit_meters)    

        warpedGray = cv2.cvtColor(warped,cv2.COLOR_RGB2GRAY)
        #use valid lines 
        driving_space = color_driving_space(warpedGray,valid_l_fit_x, valid_r_fit_x, ploty)
        img_size = (driving_space.shape[1], driving_space.shape[0])
        unwarped = cv2.warpPerspective(driving_space,invM,img_size,flags=cv2.INTER_LINEAR)    
        merged = cv2.addWeighted(image, 1, unwarped, 0.3, 0)
        add_tags(merged,valid_l_cr,valid_r_cr,valid_l_fit_x[-1], valid_r_fit_x[-1],xm_per_pix,testing=False)    

        

        f, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2, 2, figsize=(18, 7))
        f.tight_layout()
        ax1.imshow(image)
        ax1.set_title('Original image {}'.format(imNum),  fontsize=10)
        ax2.imshow(merged)
        ax2.set_title('Tagged image', fontsize=10)
        ax3.imshow(warped)
        #cv2.imwrite("test%s.jpg" % imNum, warped)
        ax3.set_title('Warped result test image {}'.format(imNum),  fontsize=10)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        ax4.imshow(lineImg1)
        ax4.set_title('Detected lanes with\n corresponding polynomial drawn', fontsize=10)
        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.05,wspace = None,hspace = 0.3)
               
        #plt.imsave('output_images/colored_driving_space.jpg',merged )
        #f.savefig('output_images/writeup_lane_boundaries.jpg')
        
    
folder_to_test = ''
folder_to_test  = 'project_video/test'
#folder_to_test ='straight_lines/test'
#folder_to_test ='challenge_video/test'
#writeup_test_pipeline_with_images(folder_to_test)

## Processing project video
run this line to process the project video

In [14]:
#define buffer size for smoothing
max_queue_size = 5
LinesDeque = collections.deque(maxlen=max_queue_size)
def process_project_video(subclip=False,begin=0,end=5):
    output='project_video_processed.mp4'
    clip1 = VideoFileClip('project_video.mp4')
    if (subclip):
        clip = clip1.fl_image(process_frame).subclip(begin,end)
    else:
        clip = clip1.fl_image(process_frame)
    %time clip.write_videofile(output, audio=False)

#process_project_video(subclip=True,begin=40,end=42)
process_project_video(subclip=False,begin=0,end=1) 

t:   0%|          | 2/1260 [00:00<01:40, 12.47it/s, now=None]

Moviepy - Building video project_video_processed.mp4.
Moviepy - Writing video project_video_processed.mp4



Moviepy - Done !
Moviepy - video ready project_video_processed.mp4
CPU times: user 5min 7s, sys: 45.6 s, total: 5min 53s
Wall time: 3min 26s


In [15]:
HTML("""
<video width="960" height="540" controls>
  <source src='project_video_processed.mp4'>
</video>
""")

## Process challenging video
please uncomment the last line to process the video

In [19]:
#define buffer size for smoothing
max_queue_size = 5
LinesDeque = collections.deque(maxlen=max_queue_size)
def process_challenging_video(subclip=False,begin=0,end=5):

    output='challenge_video_processed.mp4'
    clip1 = VideoFileClip('challenge_video.mp4')
    if (subclip):
        clip = clip1.fl_image(process_frame).subclip(begin,end)
    else:
        clip = clip1.fl_image(process_frame)
    %time clip.write_videofile(output, audio=False)

process_challenging_video(subclip=False,begin=0,end=5)

t:   0%|          | 0/485 [00:00<?, ?it/s, now=None]

Moviepy - Building video challenge_video_processed.mp4.
Moviepy - Writing video challenge_video_processed.mp4




t:  26%|██▌       | 127/485 [00:26<01:58,  3.02it/s, now=None]


t:  52%|█████▏    | 251/485 [00:55<00:38,  6.12it/s, now=None]


t:  77%|███████▋  | 375/485 [01:15<00:17,  6.38it/s, now=None]


t:  81%|████████  | 391/485 [06:27<00:13,  6.75it/s, now=None]

Moviepy - Done !
Moviepy - video ready challenge_video_processed.mp4
CPU times: user 1min 54s, sys: 19.1 s, total: 2min 13s
Wall time: 1min 36s


In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src='challenge_video_processed.mp4'>
</video>
""")

## Process harder challenging video
please uncomment the last line to process the video

In [ ]:
#define buffer size for smoothing
max_queue_size = 5
LinesDeque = collections.deque(maxlen=max_queue_size)
def process_challenging_video(subclip=False,begin=0,end=5):

    output='hard_challenge_video_processed.mp4'
    clip1 = VideoFileClip('harder_challenge_video.mp4')
    if (subclip):
        clip = clip1.fl_image(process_frame).subclip(begin,end)
    else:
        clip = clip1.fl_image(process_frame)
    %time clip.write_videofile(output, audio=False)

process_challenging_video(subclip=False,begin=0,end=5)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src='hard_challenge_video_processed.mp4'>
</video>
""")